# Selenium Webscraping Notebook 12/16/22

In [9]:
# Need this as a dependency of selenium 
# !pip install pathlib

In [10]:
# Need this as a dependency of selenium 
# !pip install ruamel-yaml

In [11]:
# Need to install selenium --> installed! 
# !pip install selenium

In [12]:
# All necessary imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from random import randint
import time
from tqdm import tqdm
from time import sleep
import json
import math
from selenium.webdriver.chrome.service import Service
import pandas as pd
import csv

In [13]:
# enabling headless mode
# Note: I'm not sure what this does 
options = Options()
options.headless = True

In [14]:
# Options() is an object within Selenium 
type(options)

selenium.webdriver.chrome.options.Options

In [15]:
# This seems to work. 
# service = Service('/usr/local/bin/chromedriver') on MacOS 
service = Service('/usr/local/bin/chromedriver')

In [16]:
driver = webdriver.Chrome(options=options, executable_path='/usr/local/bin/chromedriver')

<ipython-input-16-0ccb8b38fe74>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path='/usr/local/bin/chromedriver')


In [17]:
mainURL = "https://www.watchcollecting.com"
driver.get(mainURL)

In [19]:
# Their function for processing each auction 
def processAuction(auction):
    # driver.get(listOfAuctions[0])
    driver.get(auction)
    dataDict = {}
    for className in ['listing__statPrice', 
                    'listing__statTime', 
                    'product-subtitle', 
                    'reference-number']:

        value = driver.find_element(By.CLASS_NAME, className).get_attribute('innerHTML')
        value = re.sub('\s+', '', value).strip()
        dataDict[className] = value

    # Process reference-number
    key,value = dataDict['reference-number'].split(':')
    key = key.strip()
    value = value.strip()
    dataDict[key] = value
    del dataDict['reference-number']

    # Process listing__statPrice
    value = dataDict['listing__statPrice']
    value = re.findall('\d',value)
    try:
        dataDict['listing__statPrice'] = int(''.join(value))
    except:
        print("Value skipped", value)
        dataDict['listing__statPrice'] = 0
        # Kevin had to make change here to try-except because some of the values didn't have prices 
        # dataDict['listing__statPrice'] = int(float((''.join(value))))


    for element in driver.find_elements(By.CLASS_NAME, 'column'):
        for detail in driver.find_elements(By.TAG_NAME, 'li'):
            attribute = detail.get_attribute('innerHTML')
            if attribute[:27] == '<span class="column-title">':
                parsedSpan = attribute[27:].split('</span>')
                # <span class="column-title">Box:</span>Yes 
                key = parsedSpan[0][:-1]
                value = parsedSpan[-1].strip()
                # value is until the next tag starting, if exist
                value = value.split('<')[0]
                key = re.sub('\s+', '', key).strip()
                value = re.sub('\s+', '', value).strip()
                dataDict[key] = value

    # print(dataDict)
    return dataDict

In [20]:
# I'm assuming they are 
def getListOfDictionaries(website):
    listOfDictionaries = []
    driver.get(website)
    listOfAuctions = []
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'auction')))
    for auction in driver.find_elements(By.CLASS_NAME, 'auction'):
        listOfAuctions.append(auction.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))

    for auction in listOfAuctions:
        # This function calls the processAuction function 
        listOfDictionaries.append(processAuction(auction))

    return listOfDictionaries

In [23]:
# Try out function: 
# Doesn't work for this website of individual listing: 
# getListOfDictionaries('https://www.watchcollecting.com/for-sale/2021-rolex-sky-dweller-15')
# It works when you pass in the high-level website name and returns a list of each auction as a dictionary: 
auction_dicts = getListOfDictionaries('https://www.watchcollecting.com/')

In [25]:
# There are 24 auctions in the dict - where are these coming from? 
len(auction_dicts)

24

In [26]:
auction_dicts

[{'listing__statPrice': 6600,
  'listing__statTime': '43:32:51',
  'product-subtitle': 'SantosDeCartier',
  'Model': 'WHSA0007',
  'Box': 'Yes',
  'Papers': 'Yes',
  'Age': '2021',
  'Movement': 'automatic',
  'ConditionGrade': 'A',
  'CaseSize': '39.7mm',
  'Case': 'StainlessSteel',
  'Dial': 'Skeleton',
  'Bracelet': 'StainlessSteel',
  'LOT': '#004602',
  'Location': 'London,UnitedKingdom',
  'Seller': '',
  'Currency': 'GBP'},
 {'listing__statPrice': 14500,
  'listing__statTime': '43:37:49',
  'product-subtitle': 'Daytona',
  'Model': '116520',
  'Box': 'Yes',
  'Papers': 'Yes',
  'Age': '2009',
  'Movement': 'automatic',
  'ConditionGrade': 'AAA',
  'CaseSize': '40mm',
  'Case': 'StainlessSteel',
  'Dial': 'WhiteBaton',
  'Bracelet': 'StainlessSteelOyster',
  'LOT': '#004603',
  'Location': 'London,UnitedKingdom',
  'Seller': '',
  'Currency': 'GBP'},
 {'listing__statPrice': 6200,
  'listing__statTime': '43:42:48',
  'product-subtitle': 'Datejust',
  'Model': '278274',
  'Box': 'Y